# Extending the gstlearn

The aim of this tutorial is to show how to customize the default behaviour of the **gstlearn** package. The choosen example here shows how to define your own moving neighborhood criteria.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Initialization

In [ ]:
import pandas as pd
import numpy as np
import gstlearn as gl
import gstlearn.plot as gp
import urllib.request

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Temperatures.csv'
filecsv, head = urllib.request.urlretrieve(url)
datcsv = pd.read_csv(filecsv, na_values="MISS")
datcsv

We create a Db and set the **locators** for the coordinates and the variable of interest. We then create a selection to mask automatically undefined values for the temperature variable.

In [ ]:
data = gl.Db()
for field in datcsv.columns :
  data[field] = datcsv[field]
data.setLocators(["Longitude","Latitude"], gl.ELoc.X)
data.setLocator("January_temp", gl.ELoc.Z)
# Selection for defined values only
data["TempOK"] = ~np.isnan(data["January_temp"])
data.setLocator("TempOK", gl.ELoc.SEL)
data

In [ ]:
gp.plot(data,sizmin=5,sizmax=50,flagLegendSymbol=True)

We create automatically a grid data base covering all the data points. Number of node is choosen by trial/error in order to have more or less an homogeneous mesh size.

In [ ]:
grid = gl.DbGrid.createCoveringDb(data, nx=[80,130], margin=[50,50])
grid

In [ ]:
gp.plot(grid)
gp.plot(data,sizmin=5,sizmax=50,flagLegendSymbol=True)

# Playing with NeighMoving

We create a moving neighborhood definition object having only one big sector (a circle) with maximum 3 samples.

In [ ]:
nmini = 1
nmaxi = 3
radius = 300 # Big radius
nsect = 1
neigh = gl.NeighMoving.create(False, nmaxi, radius, nmini, nsect)
neigh

We trigger the moving neighborhood algorithm applied to the target node (of the grid) which rank is 4511. We obtain three neighbors and display their ranks. 

In [ ]:
node = 4511
neigh.attach(data, grid)
ranks = neigh.select(node)
ranks # Ranks 23, 75 and 143

Here is a small function using ax.neigh gstlearn function that offer the capability to display neigh results. The target node is in black and the neighbors in blue.

In [ ]:
def show_neigh(data, grid, neigh, node, ranks):
  ax = data.plot(name_size="January_temp",sizmax=30, useSel=True)
  if (len(ranks) > 0):
    dataSel = data.clone()
    dataSel.addSelectionByRanks(ranks)
    ax = dataSel.plot(color='blue',name_size="January_temp",sizmax=30)
  ax.neigh(neigh, grid, node)
  ax.decoration("Standard Neighborhood")
    
show_neigh(data, grid, neigh, node, ranks)

We reduce the radius of the circle so that only one neighbor will be selected.

In [ ]:
nmini = 1
nmaxi = 3
radius = 30 # Smaller radius
nsect = 1
neigh = gl.NeighMoving.create(False, nmaxi, radius, nmini, nsect)
neigh

In [ ]:
node = 4511
neigh.attach(data, grid)
ranks = neigh.select(node)
ranks # Only rank 143

In [ ]:
show_neigh(data, grid, neigh, node, ranks)

# Extending the gstlearn

In this pargraph, we want to add our own neighborhood "checker" by defining it in Python. We choose to mimick the maximum radius checker which is already available in the standard *NeighMoving* class.

We create a new class (in Python) that inherits from the **gstlearn** C++ class named *ABiTargetCheck*. In that class, we define a member called *radius* and override two abstract methods:
 - isOK: that returns *True* if two space targets (see SpaceTarget class) are considered in the same neighborhood.
 - toString: that returns the characters string describing the class content.

In [ ]:
class BiTargetCheckMaxRadius(gl.ABiTargetCheck):
    def __init__(self, radius):
        super(BiTargetCheckMaxRadius,self).__init__()
        self.radius = radius
        
    def isOK(self,st1,st2):
        return st1.getDistance(st2) < self.radius
    
    def toString(self,strfmt):
        return "This is my Max Radius checker: {}".format(self.radius)


Then, we recreate a NeighMoving object having a big radius (should select 3 neighbors), but we add our customized checker which indicates a small radius. Doing that, we hope that the neighborhood algorithm will select back only one sample (with rank 143).

In [ ]:
nmini = 1
nmaxi = 3
radius = 300 # Big radius
neigh2 = gl.NeighMoving.create(False, nmaxi=nmaxi, nmini=nmini,radius=radius)
# Prefer my own maximum radius checker!
btc = BiTargetCheckMaxRadius(30)
neigh2.addBiTargetCheck(btc)
neigh2.display()

In [ ]:
node = 4511
neigh2.attach(data, grid)
ranks = neigh2.select(node)
ranks # Should return 143

Here, the big ellipsoid circle is shown (default behavior) but only one sample is selected due to our customized checker.

In [ ]:
show_neigh(data, grid, neigh2, node, ranks)